## NeuroAlign - Inference

This notebook can be used to predict alignments from fasta files of unaligned sequences using a pretrained NeuroAlign model.

NeuroAlign is a deep learning based end-to-end model that is able to construct multiple sequence alignments by supervised learning of reference alignments. Our model maps raw sequences
\- for instance over the standard aminoacid alphabet but not limited to that \- to a soft alignment represented by a membership probablity for each residuum and each alignment column in a (dynamically sized) consensus sequence. NeuroAlign can also output representation vectors for the alignment columns which encode the respective distribution of aminoacids that can be useful for higher level learning tasks.

The weight files provided with this notebook are trained on Pfam. We used all 18259 protein families available at the time of our access, but only their respective seed alignments.

### Requirements

This notebook requires tensorflow (2.1 or higher) and matplotlib. Other than that, it is self-contained. 

Model weight files are provided with github under ./models.

If a GPU is present, the notebook will automatically attempt to use it. 

## Setting up the model

We provide configurated and pretrained "out of the box" models. All what has to be done is to load a model using its identifier.

NeuroAlign maps sequences to a consensus by iteratively transforming each sequence to consensus contributions individually and aggregating information over the number of sequences thereafter.

More precisely, at each iteration $t$ we have sequence states $S^t_{i,j}$ which represent the $j$-th residue of sequence $i$ and column contributions $C^t_{i,l}$ corresponding to the contribution of sequence $i$ to alignment column $l$. We use a combination of LSTM (for positional encoding) and Transformer to model for each sequence $i$ which residues $j$ correspond to which column $l$. While this approach can learn strong residual level features for each sequence independently, we need a way of letting the sequences interact while still scaling linearly in the number of sequences.

We update $C^t_{i,l}$ in a way that is inspired by Graph Neural Networks (GNNs). That is, we pass messages to allow information flow between sequences but avoid quadratic scaling by aggregating over the sequence dimension:

- compute messages: $ M^t_{i,l} = \phi_M (C^t_{i,l}) $
- aggregate messages over the sequence dimension and exclude self loops: $ \bar{M}^t_{-i,l} = \sum_{i' \neq i}  M^t_{i', l} $
- update column contributions $C^{t+1}_{i,l} = \phi_U (C^t_{i,l}, \bar{M}^t_{-i,l})$

This is similar to the behavior of a GNN on a graph, where each column contribution is connected to an (implicitely existing) column node that aggregates information over all sequences and passes this condensed information back to all column contribution states.

While this aproach allows for column-level information flow between sequences, the features can not adopt to different distances of the respective sequence to the consensus (in the aggregation, they are weighted all the same). In biology such an effect might be crutial, because input sequences might be distantly or closely related to a consensus and their contribution should differ depending on that distance. Therefore, we refine above procedure by introducing global sequence and global consensus summaries:

- compute (masked) sequence summaries: $ \bar{S}^t_i = \sum_{j=1}^{|S_i|} \phi_S (S^t_{i,j}) $
- compute contribution summaries up to column $l$: $ \bar{C}^t_{i,l} = \sum^l_{l'=1} \phi_G (C^t_{i,l'}) $
- compute a global summary up to column $l$: $ G^t_l = \sum_i \bar{C}^t_{i,l} $
- change the update step above to: $C^{t+1}_{i,l} = \phi_U (C^t_{i,l}, \bar{M}^t_{-i,l}, \bar{S}^t_i, G^t_l)$

We would not call $\bar{C}^t_i$ and $G^t$ "states" or "representations" as they are mere summaries of sequences and consensus. To simplify the model, we do not have a persistent state that exists across iterations $t$ for either of these.

In [1]:
#import sys
#!{sys.executable} -m pip install tensorflow_probability==0.11.0

import Model as model

neuroalign, neuroalign_config = model.make_neuro_align_model("cross_transfo3")

Successfully loaded weights: cross_transfo3 model


## Loading a fasta file
NeuroAlign operates on raw sequences of aminoacids in fasta file format. Sequences can come in any number and order. The model is inherently invariant to permuntations of the sequences. 

We first parse the fasta files and convert to a numeric "one-hot" format which can be passed to the NeuroAlign model. 

In [2]:
import sys
import numpy as np
import Data as data

#######################################################################################
################ you may modify these variables to test NeuroAlign ####################
#######################################################################################

file = "./data/BB11001.fasta"
#file = "../brain/Pfam/alignments/PF00001.fasta"

# set this to True, if the fasta file is already aligned 
# i.e. it contains gap symbols and all sequences have the same length
# if True, the gap symbols will be automatically stripped away before inference and will
# only be used for evaluating the model output
# if False, the model will just compute its output and skip all evaluation
has_gaps = True 

#######################################################################################
#######################################################################################
#######################################################################################

fasta_file = data.Fasta(file, gaps = has_gaps, contains_lower_case = True)

if not fasta_file.valid:
    sys.exit("Invalid fasta file.")

selected_sequences = list(range(len(fasta_file.raw_seq))) #all and in order
if has_gaps:
    input_dict, target_dict = data.get_input_target_data(fasta_file, selected_sequences, neuroalign_config)
else:
    seq = fasta_file.one_hot_sequences(selected_sequences)
    input_dict = {"sequences" : seq}
    
print("Successfully loaded file:", file)

Successfully loaded file: ./data/BB11001.fasta


In [24]:
#print("Alphabet:",list(enumerate(data.ALPHABET)))

#seq_i = 0
#print("begin of sequence "+str(seq_i)+":")
#print("$" + fasta_file.aminoacid_seq_str(seq_i)[:4]) #each sequence is preceeded by a gap dummy "_" followed by the start-marker $
#print(input_dict["sequences"][seq_i,:6])

#col_i = 3
#if has_gaps:
    #print("column "+str(col_i)+":")
    #print(fasta_file.column_str(col_i))
    #print(target_dict["out_gaps"][:,0])
col=2
print(input_dict["sequences_residual_mask"][:,col,:10])

print(target_dict["out_gaps"][:,col])
neuroalign(input_dict)[:,col]

[[0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]


<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[5.8879524e-01, 4.1106844e-01, 3.4419456e-13, 1.3631961e-04],
       [5.7190299e-01, 4.2804417e-01, 8.2399209e-14, 5.2819287e-05],
       [2.8741587e-07, 9.9999940e-01, 5.3684466e-13, 3.2412783e-07],
       [5.8970308e-01, 4.1022450e-01, 1.5469380e-13, 7.2461284e-05]],
      dtype=float32)>

NeuroAlign is an autoregressive model that predicts a consensus sequence of columns one at a time conditioned on the input sequences and all previous columns. Its decision for the aminoacid distribution of a column is based on all sequences and all previously predicted columns. Inference internally starts with a consensus consisting only of a special "column start" marker. It is then rolled out autoregressively in order to generate columns until a "column end" marker is predicted or a maximal length (default 1000) is reached.

We provide a method for generating the columns this way.

In [4]:
# generates a 2D tensor "column_distributions" with shape [num_columns, alphabet_size]
# and a 3D tensor "attention_weights" with shape [num_seq, len_seq, num_cols]
attention_weights = model.gen_columns(input_dict["sequences"], 
                                      np.array(fasta_file.seq_lens),
                                      neuroalign, neuroalign_config)

print("Prediction run on file", fasta_file.filename, "with", len(fasta_file.raw_seq), 
      "sequences of average length", sum(fasta_file.seq_lens) / len(fasta_file.raw_seq), ".")
print("Predicted a soft alignment with", column_distributions.shape[0], "columns.")
if has_gaps:
    print("The reference has", fasta_file.alignment_len, "columns.")

(4, 93, 28)
(4, 1, 4)
tf.Tensor(
[[8.53040516e-02 9.14655685e-01 1.61536501e-13 4.02715086e-05]
 [2.13290807e-02 9.78656411e-01 3.45680515e-14 1.45449139e-05]
 [3.71613614e-02 9.62825239e-01 1.48504324e-14 1.34766815e-05]
 [2.74455827e-02 9.72534716e-01 9.56133095e-14 1.96922283e-05]], shape=(4, 4), dtype=float32)


UnboundLocalError: local variable 'indices' referenced before assignment

## Evaluation

So far, a soft alignment was predicted and we seek to evaluate it. NeuroAlign uses an attention mechanism that measures the degree of importance of a residuum to a particular column. A good way to visualize the prediction is to plot the attention weights. This way, we can make a visual comparison of the reference alignment with the predicted attention scores. NeuroAlign is trained in a way such that as much homologous residues from the reference are aligned in the prediction as possible. Thus, they may share a column in the prediction that has a different index in the reference.

Furthermore, we need a way to evaluate the models performance in terms of numeric metrics. To that end, we compute precision and recall for all binary variables "residue X and residue Y are aligned".

In [ ]:
import Evaluation as evl
import numpy as np

#for comparison compute the output with teacher forcing enabled
#i.e. when predicting column i the model knowns how reference columns 1...i-1 look like
#teacher_forced_column_distributions, teacher_forced_attention_weights = neuroalign(input_dict)
teacher_forced_gaps = neuroalign(input_dict)

#print(teacher_forced_gaps[:,:4])

def flip(x):
    return np.transpose(x, [0,2,1])

evl.plot_memberships(attention_weights, [attention_weights], ["Output", "Teacher forced output"])

p = evl.precision(target_dict["out_attention"], attention_weights).numpy()
r = evl.recall(target_dict["out_attention"], attention_weights).numpy()

print("alignment precision: ", p)
print("alignment recall: ", r)


p = evl.precision(target_dict["out_attention"], teacher_forced_attention_weights).numpy()
r = evl.recall(target_dict["out_attention"], teacher_forced_attention_weights).numpy()

print("alignment precision (teacher forced): ", p)
print("alignment recall (teacher forced): ", r)


print("teacher forced KLD: ", evl.kld(target_dict["out_columns"], teacher_forced_column_distributions).numpy())
alphabet = data.ALPHABET + ["_", "$", "#"]
for i in range(len(alphabet)):
    l = evl.one_out_kld(i)(target_dict["out_columns"], teacher_forced_column_distributions).numpy()
    print("teacher forced one out KLD for", alphabet[i], ":", l)